In [ ]:
!pip install tensorflow

In [ ]:
pip install keras

# Import Movie Review Data

Set the seed

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np

In [ ]:
np.random.seed(42)

Import the dataset as pandas dataframe

In [ ]:
import pandas as pd

In [ ]:
your_local_path="D:/Deep_Learning/LSTM"

Data can be downloaded from Kaggle at the following URL

- https://www.kaggle.com/c/word2vec-nlp-tutorial/data

In [ ]:
df = pd.read_csv("D:/Deep_Learning/LSTM/labeledTrainData.tsv",header=0, delimiter="\t", quoting=3)


Split Data into Training and Test Data

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df['review'],
    df['sentiment'],
    test_size=0.2, 
    random_state=42
)

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

# Build the Tokenizer

In [ ]:
top_words = 10000

In [ ]:
t = Tokenizer(num_words=top_words) # num_words -> Vocablury size

In [ ]:
t.fit_on_texts(X_train.tolist())

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [ ]:
X_train = t.texts_to_sequences(X_train.tolist())

In [ ]:
X_test = t.texts_to_sequences(X_test.tolist())

How many words in each review?

# Pad Sequences - Important

In [ ]:
from tensorflow.python.keras.preprocessing import sequence

In [ ]:
max_review_length = 300

In [ ]:
X_train = sequence.pad_sequences(X_train,maxlen=max_review_length,padding='post')

In [ ]:
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length, padding='post')

# Build Embedding Matrix from Pre-Trained Word2Vec

Load pre-trained Gensim Embeddings

In [ ]:
import gensim

In [ ]:
word2vec = gensim.models.Word2Vec.load('word2vec-movie-50')

Embedding Size

In [ ]:
embedding_vector_length = word2vec.wv.syn0.shape[1]

Build matrix for current data

In [ ]:
embedding_matrix = np.zeros((top_words + 1, embedding_vector_length))

In [ ]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > top_words:
        break
    if word in word2vec.wv.vocab:
        embedding_vector = word2vec.wv[word]
        embedding_matrix[i] = embedding_vector

# Build the Graph

In [ ]:
from tensorflow.python.keras.models import Sequential

In [ ]:
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM

In [ ]:
model = Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [ ]:
model.add(Embedding(top_words + 1,
                    embedding_vector_length,
                    input_length=max_review_length,
                   weights=[embedding_matrix],
                   trainable=False)
         )

Embedding Layer Output - 
[Batch_Size , Review Length , Embedding_Size]

Add Layer with 100 LSTM Memory Units

In [ ]:
model.add(dropout=0.2)

In [ ]:
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
#outout is last hidden state
#batch_size, 100

In [ ]:
model.add(dropout=0.2)

In [ ]:
model.add(Dense(1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

# Execute the graph

In [ ]:
model.fit(X_train,y_train,
          epochs=30,
          batch_size=128,          
          validation_data=(X_test, y_test),
         verbose=1)